In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import simpy
from collections import defaultdict

class CSMA_CA_Simulator:
    def __init__(self):
        # Simulation parameters
        self.RECEIVING_PROCESSING_TIME = 1
        self.SENDING_NODE_INVOKE_TIME = 0
        self.SIM_TIME = 20000
        
        # Network parameters
        self.NUMBER_OF_NODES = 4
        self.COVERAGE_RANGE = 1
        self.COVERAGE_NODES_IDLE = []
        
        # CSMA/CA parameters
        self.ATTEMPT_LIMIT = 10
        self.DIFS = 5
        self.CONTENTION_WINDOW_SLOT = 1
        self.SIFS = 2
        self.ACK_TIMER = 2
        self.CARRIER_SENSING_DURATION = 2
        
        # Protocol options
        self.CTS_RTS = True
        
        # Statistics
        self.PACKETS = 0
        self.COLLISIONS = 0
        self.PREVIOUS_COLLISION_TIME = -1
        self.LAST_TIME = 0
        self.Data_senders = 0
        
        # Initialize coverage nodes
        self._init_coverage_nodes()
        
        # Channel resource
        self.channel = None
        
    def _init_coverage_nodes(self):
        """Initialize coverage nodes status"""
        self.COVERAGE_NODES_IDLE = [
            True for _ in range((self.NUMBER_OF_NODES // self.COVERAGE_RANGE) + 1)
        ]
    
    def reset_stats(self):
        """Reset simulation statistics"""
        self.PACKETS = 0
        self.COLLISIONS = 0
        self.PREVIOUS_COLLISION_TIME = -1
        self.LAST_TIME = 0
        self.Data_senders = 0
        self._init_coverage_nodes()
    
    class ReceivingNode:
        def __init__(self, env, simulator):
            self.env = env
            self.simulator = simulator
            self.processing_time = simulator.RECEIVING_PROCESSING_TIME
            self.packetType = -1
            self.request = None
        
        def receive(self, sending_node):
            global Data_senders
            
            # Transmission time
            yield self.env.timeout(self.processing_time / 2)
            self.simulator.Data_senders += 1
            yield self.env.timeout(self.processing_time / 2)
            
            if self.simulator.Data_senders > 1:
                print(f"COLLISION! GARBAGE DATA RECEIVED AT ROUTER at {self.env.now}")
                if self.simulator.PREVIOUS_COLLISION_TIME != self.env.now:
                    self.simulator.COLLISIONS += 1
                    self.simulator.PREVIOUS_COLLISION_TIME = self.env.now
                yield self.env.timeout(self.processing_time)
            else:
                if self.packetType == 0:
                    print(f"ROUTER RECEIVED RTS FROM Node {sending_node} at {self.env.now}")
                    self.request = self.simulator.channel.request()
                    yield self.request
                else:
                    print(f"ROUTER RECEIVED DATA FRAME FROM Node {sending_node} at {self.env.now}")
            
            # Receiving time
            self.simulator.Data_senders -= 1
            yield self.env.timeout(self.processing_time)
        
        def release_request(self):
            if self.request is not None:
                self.simulator.channel.release(self.request)
                self.request = None
    
    class Node:
        def __init__(self, env, simulator):
            self.env = env
            self.simulator = simulator
            self.attempt = 0
            self.R = 0
        
        def sending_node(self, name, receiving_node):
            while self.attempt < self.simulator.ATTEMPT_LIMIT:
                # Checking for idle channel
                if self.simulator.CTS_RTS:
                    request = self.simulator.channel.request()
                    yield request
                    self.simulator.channel.release(request)
                
                while not self._get_local_sensing(name):
                    yield self.env.timeout(self.simulator.CARRIER_SENSING_DURATION)
                
                # Waiting for DIFS
                if self.simulator.CTS_RTS:
                    request = self.simulator.channel.request()
                    yield request
                    self.simulator.channel.release(request)
                
                yield self.env.timeout(random.randint(
                    self.simulator.DIFS, 2 * self.simulator.DIFS))
                
                # Choose random backoff
                self.R = random.randint(0, 2 ** min(self.attempt, 10))  # Cap exponential growth
                contention_window = self.R * self.simulator.CONTENTION_WINDOW_SLOT
                
                # Wait for backoff slots
                for _ in range(self.R):
                    if self.simulator.CTS_RTS:
                        request = self.simulator.channel.request()
                        yield request
                        self.simulator.channel.release(request)
                    
                    while not self._get_local_sensing(name):
                        yield self.env.timeout(self.simulator.CARRIER_SENSING_DURATION)
                    
                    if self.simulator.CTS_RTS:
                        request = self.simulator.channel.request()
                        yield request
                        self.simulator.channel.release(request)
                    
                    yield self.env.timeout(contention_window)
                
                # Final channel check before transmission
                if self.simulator.CTS_RTS:
                    request = self.simulator.channel.request()
                    yield request
                    self.simulator.channel.release(request)
                
                while not self._get_local_sensing(name):
                    yield self.env.timeout(self.simulator.CARRIER_SENSING_DURATION)
                
                # RTS/CTS mechanism
                if self.simulator.CTS_RTS:
                    # Check for CTS
                    request = self.simulator.channel.request()
                    yield request
                    self.simulator.channel.release(request)
                    
                    T_rts = float(self.env.now)
                    print(f"RTS SENT BY NODE {name} at {T_rts}")
                    receiving_node.packetType = 0
                    self._set_local_sensing(name, False)
                    yield self.env.process(receiving_node.receive(name))
                    self._set_local_sensing(name, True)
                    
                    T_cts = float(self.env.now)
                    
                    if T_cts - T_rts > self.simulator.SIFS:
                        # Collision occurred
                        self._handle_collision(name)
                        yield self.env.timeout(self.R * self.simulator.RECEIVING_PROCESSING_TIME)
                    else:
                        # CTS received successfully
                        print(f"CTS RECEIVED BY NODE {name} at {T_cts}")
                        yield self.env.timeout(self.simulator.SIFS)
                        
                        # Send data frame
                        T_send_frame = float(self.env.now)
                        receiving_node.packetType = 1
                        print(f"FRAME SENT BY NODE {name} at {T_send_frame}")
                        
                        self._set_local_sensing(name, False)
                        yield self.env.process(receiving_node.receive(name))
                        self._set_local_sensing(name, True)
                        
                        T_receive_frame = float(self.env.now)
                        
                        if T_receive_frame - T_send_frame <= self.simulator.ACK_TIMER:
                            # Successful transmission
                            print(f"ACK RECEIVED BY NODE {name} at {T_receive_frame}")
                            self.simulator.LAST_TIME = T_receive_frame
                            self.simulator.PACKETS += 1
                            receiving_node.release_request()
                            break
                        else:
                            # Collision occurred
                            self._handle_collision(name)
                            yield self.env.timeout(self.R * self.simulator.RECEIVING_PROCESSING_TIME)
                else:
                    # Without RTS/CTS
                    T_send_frame = float(self.env.now)
                    print(f"FRAME SENT BY NODE {name} at {T_send_frame}")
                    receiving_node.packetType = 1
                    
                    self._set_local_sensing(name, False)
                    yield self.env.process(receiving_node.receive(name))
                    self._set_local_sensing(name, True)
                    
                    T_receive_frame = float(self.env.now)
                    
                    if T_receive_frame - T_send_frame <= self.simulator.ACK_TIMER:
                        # Successful transmission
                        print(f"ACK RECEIVED BY NODE {name} at {T_receive_frame}")
                        self.simulator.LAST_TIME = T_receive_frame
                        self.simulator.PACKETS += 1
                        break
                    else:
                        # Collision occurred
                        self._handle_collision(name)
                        yield self.env.timeout(self.R * self.simulator.RECEIVING_PROCESSING_TIME)
        
        def _handle_collision(self, name):
            """Handle collision by incrementing attempt counter"""
            self.attempt += 1
            print(f"NODE {name} AT ATTEMPT k={self.attempt}, BACKOFF TIME={self.R * self.simulator.RECEIVING_PROCESSING_TIME} at {self.env.now}")
        
        def _get_local_sensing(self, name):
            """Get local sensing result for the node"""
            return self.simulator.COVERAGE_NODES_IDLE[
                (name - 1) // self.simulator.COVERAGE_RANGE]
        
        def _set_local_sensing(self, name, value):
            """Set local sensing result for the node"""
            self.simulator.COVERAGE_NODES_IDLE[
                (name - 1) // self.simulator.COVERAGE_RANGE] = value
    
    def setup(self, env):
        """Setup the simulation environment"""
        receiving_node = self.ReceivingNode(env, self)
        sending_node_name = 0
        
        while sending_node_name < self.NUMBER_OF_NODES:
            yield env.timeout(self.SENDING_NODE_INVOKE_TIME)
            sending_node_name += 1
            print(f"NODE {sending_node_name} COMES AT {env.now:.2f}: WAITING FOR IDLE")
            node = self.Node(env, self)
            env.process(node.sending_node(sending_node_name, receiving_node))
    
    def simple_run(self):
        """Run a simple simulation"""
        self.reset_stats()
        env = simpy.Environment()
        self.channel = simpy.Resource(env, 1)
        
        env.process(self.setup(env))
        env.run(until=self.SIM_TIME)
        
        print(f"TOTAL PACKETS TRANSFERRED: {self.PACKETS}")
        print(f"TOTAL COLLISIONS: {self.COLLISIONS}")
        print(f"THROUGHPUT: {self.PACKETS / self.SIM_TIME if self.SIM_TIME > 0 else 0}")
    
    def analyze_cts_rts(self, max_nodes=30, iterations=30):
        """Analyze performance with and without RTS/CTS"""
        print("ANALYZING RTS/CTS PERFORMANCE...")
        
        results = {
            'with_rts': defaultdict(list),
            'without_rts': defaultdict(list)
        }
        
        original_cts_rts = self.CTS_RTS
        original_nodes = self.NUMBER_OF_NODES
        
        # Test without RTS/CTS
        self.CTS_RTS = False
        for nodes in range(1, max_nodes + 1):
            print(f"\nTesting without RTS/CTS with {nodes} nodes")
            self.NUMBER_OF_NODES = nodes
            
            throughputs = []
            for i in range(iterations):
                self.reset_stats()
                env = simpy.Environment()
                self.channel = simpy.Resource(env, 1)
                env.process(self.setup(env))
                env.run(until=self.SIM_TIME)
                
                if self.LAST_TIME > 0:
                    throughput = self.PACKETS / self.LAST_TIME
                else:
                    throughput = 0
                throughputs.append(throughput)
            
            avg_throughput = sum(throughputs) / iterations
            results['without_rts'][nodes] = avg_throughput
            print(f"Average throughput: {avg_throughput}")
        
        # Test with RTS/CTS
        self.CTS_RTS = True
        for nodes in range(1, max_nodes + 1):
            print(f"\nTesting with RTS/CTS with {nodes} nodes")
            self.NUMBER_OF_NODES = nodes
            
            throughputs = []
            for i in range(iterations):
                self.reset_stats()
                env = simpy.Environment()
                self.channel = simpy.Resource(env, 1)
                env.process(self.setup(env))
                env.run(until=self.SIM_TIME)
                
                if self.LAST_TIME > 0:
                    throughput = self.PACKETS / self.LAST_TIME
                else:
                    throughput = 0
                throughputs.append(throughput)
            
            avg_throughput = sum(throughputs) / iterations
            results['with_rts'][nodes] = avg_throughput
            print(f"Average throughput: {avg_throughput}")
        
        # Restore original settings
        self.CTS_RTS = original_cts_rts
        self.NUMBER_OF_NODES = original_nodes
        
        # Plot results
        plt.figure(figsize=(12, 6))
        nodes_list = list(range(1, max_nodes + 1))
        
        plt.plot(nodes_list, [results['without_rts'][n] for n in nodes_list], 
                'r-', label='Without RTS/CTS')
        plt.plot(nodes_list, [results['with_rts'][n] for n in nodes_list], 
                'b-', label='With RTS/CTS')
        
        plt.xlabel("Number of Nodes")
        plt.ylabel("Throughput (packets/unit time)")
        plt.title("Throughput Comparison: With vs Without RTS/CTS")
        plt.legend()
        plt.grid(True)
        plt.show()
        
        return results
    
    def analyze_collisions(self, max_nodes=30, iterations=30):
        """Analyze collision behavior with and without RTS/CTS"""
        print("ANALYZING COLLISION BEHAVIOR...")
        
        results = {
            'with_rts': defaultdict(list),
            'without_rts': defaultdict(list)
        }
        
        original_cts_rts = self.CTS_RTS
        original_nodes = self.NUMBER_OF_NODES
        
        # Test without RTS/CTS
        self.CTS_RTS = False
        for nodes in range(1, max_nodes + 1):
            print(f"\nTesting without RTS/CTS with {nodes} nodes")
            self.NUMBER_OF_NODES = nodes
            
            collisions = []
            for i in range(iterations):
                self.reset_stats()
                env = simpy.Environment()
                self.channel = simpy.Resource(env, 1)
                env.process(self.setup(env))
                env.run(until=self.SIM_TIME)
                collisions.append(self.COLLISIONS)
            
            avg_collisions = sum(collisions) / iterations
            results['without_rts'][nodes] = avg_collisions
            print(f"Average collisions: {avg_collisions}")
        
        # Test with RTS/CTS
        self.CTS_RTS = True
        for nodes in range(1, max_nodes + 1):
            print(f"\nTesting with RTS/CTS with {nodes} nodes")
            self.NUMBER_OF_NODES = nodes
            
            collisions = []
            for i in range(iterations):
                self.reset_stats()
                env = simpy.Environment()
                self.channel = simpy.Resource(env, 1)
                env.process(self.setup(env))
                env.run(until=self.SIM_TIME)
                collisions.append(self.COLLISIONS)
            
            avg_collisions = sum(collisions) / iterations
            results['with_rts'][nodes] = avg_collisions
            print(f"Average collisions: {avg_collisions}")
        
        # Restore original settings
        self.CTS_RTS = original_cts_rts
        self.NUMBER_OF_NODES = original_nodes
        
        # Plot results
        plt.figure(figsize=(12, 6))
        nodes_list = list(range(1, max_nodes + 1))
        
        plt.plot(nodes_list, [results['without_rts'][n] for n in nodes_list], 
                'r-', label='Without RTS/CTS')
        plt.plot(nodes_list, [results['with_rts'][n] for n in nodes_list], 
                'b-', label='With RTS/CTS')
        
        plt.xlabel("Number of Nodes")
        plt.ylabel("Number of Collisions")
        plt.title("Collision Comparison: With vs Without RTS/CTS")
        plt.legend()
        plt.grid(True)
        plt.show()
        
        return results
    
    def analyze_coverage_area(self, max_coverage=30, iterations=30):
        """Analyze performance with different coverage areas"""
        print("ANALYZING COVERAGE AREA IMPACT...")
        
        results = {
            'with_rts': defaultdict(list),
            'without_rts': defaultdict(list)
        }
        
        original_cts_rts = self.CTS_RTS
        original_coverage = self.COVERAGE_RANGE
        self.NUMBER_OF_NODES = 10  # Fixed number of nodes for this analysis
        
        # Test without RTS/CTS
        self.CTS_RTS = False
        for coverage in range(1, max_coverage + 1):
            print(f"\nTesting without RTS/CTS with coverage {coverage}")
            self.COVERAGE_RANGE = coverage
            
            throughputs = []
            for i in range(iterations):
                self.reset_stats()
                env = simpy.Environment()
                self.channel = simpy.Resource(env, 1)
                env.process(self.setup(env))
                env.run(until=self.SIM_TIME)
                
                if self.LAST_TIME > 0:
                    throughput = self.PACKETS / self.LAST_TIME
                else:
                    throughput = 0
                throughputs.append(throughput)
            
            avg_throughput = sum(throughputs) / iterations
            results['without_rts'][coverage] = avg_throughput
            print(f"Average throughput: {avg_throughput}")
        
        # Test with RTS/CTS
        self.CTS_RTS = True
        for coverage in range(1, max_coverage + 1):
            print(f"\nTesting with RTS/CTS with coverage {coverage}")
            self.COVERAGE_RANGE = coverage
            
            throughputs = []
            for i in range(iterations):
                self.reset_stats()
                env = simpy.Environment()
                self.channel = simpy.Resource(env, 1)
                env.process(self.setup(env))
                env.run(until=self.SIM_TIME)
                
                if self.LAST_TIME > 0:
                    throughput = self.PACKETS / self.LAST_TIME
                else:
                    throughput = 0
                throughputs.append(throughput)
            
            avg_throughput = sum(throughputs) / iterations
            results['with_rts'][coverage] = avg_throughput
            print(f"Average throughput: {avg_throughput}")
        
        # Restore original settings
        self.CTS_RTS = original_cts_rts
        self.COVERAGE_RANGE = original_coverage
        
        # Plot results
        plt.figure(figsize=(12, 6))
        coverage_list = list(range(1, max_coverage + 1))
        
        plt.plot(coverage_list, [results['without_rts'][c] for c in coverage_list], 
                'r-', label='Without RTS/CTS')
        plt.plot(coverage_list, [results['with_rts'][c] for c in coverage_list], 
                'b-', label='With RTS/CTS')
        
        plt.xlabel("Coverage Area")
        plt.ylabel("Throughput (packets/unit time)")
        plt.title("Throughput vs Coverage Area (10 Nodes)")
        plt.legend()
        plt.grid(True)
        plt.show()
        
        return results


# Main execution
if __name__ == "__main__":
    simulator = CSMA_CA_Simulator()
    
    print("Starting basic simulation...")
    simulator.simple_run()
    
    print("\nAnalyzing RTS/CTS performance...")
    simulator.analyze_cts_rts()
    
    print("\nAnalyzing collision behavior...")
    simulator.analyze_collisions()
    
    print("\nAnalyzing coverage area impact...")
    simulator.analyze_coverage_area()

In [ ]:
class EnhancedAnalyzer:
    def __init__(self, simulator):
        self.simulator = simulator
    
    def comprehensive_analysis(self, max_nodes=30, iterations=30):
        """Run a comprehensive analysis with multiple metrics"""
        print("RUNNING COMPREHENSIVE ANALYSIS...")
        
        metrics = {
            'throughput': {'with_rts': [], 'without_rts': []},
            'collisions': {'with_rts': [], 'without_rts': []},
            'latency': {'with_rts': [], 'without_rts': []},
            'fairness': {'with_rts': [], 'without_rts': []},
            'efficiency': {'with_rts': [], 'without_rts': []},
            'success_rate': {'with_rts': [], 'without_rts': []}
        }
        
        node_counts = list(range(1, max_nodes + 1))
        
        # Test with and without RTS/CTS
        for use_rts in [False, True]:
            self.simulator.CTS_RTS = use_rts
            key = 'with_rts' if use_rts else 'without_rts'
            
            for nodes in node_counts:
                print(f"\nTesting {key} with {nodes} nodes")
                self.simulator.NUMBER_OF_NODES = nodes
                
                throughputs = []
                collisions = []
                latencies = []
                fairness_indices = []
                efficiencies = []
                success_rates = []
                
                for _ in range(iterations):
                    self.simulator.reset_stats()
                    env = simpy.Environment()
                    self.simulator.channel = simpy.Resource(env, 1)
                    env.process(self.simulator.setup(env))
                    env.run(until=self.simulator.SIM_TIME)
                    
                    # Calculate metrics
                    if self.simulator.LAST_TIME > 0:
                        throughput = self.simulator.PACKETS / self.simulator.LAST_TIME
                        latency = self.simulator.LAST_TIME / self.simulator.PACKETS if self.simulator.PACKETS > 0 else 0
                        efficiency = throughput / (self.simulator.NUMBER_OF_NODES * 0.1)  # Simplified efficiency metric
                        success_rate = self.simulator.PACKETS / (self.simulator.PACKETS + self.simulator.COLLISIONS) if (self.simulator.PACKETS + self.simulator.COLLISIONS) > 0 else 0
                    else:
                        throughput = 0
                        latency = 0
                        efficiency = 0
                        success_rate = 0
                    
                    throughputs.append(throughput)
                    collisions.append(self.simulator.COLLISIONS)
                    latencies.append(latency)
                    efficiencies.append(efficiency)
                    success_rates.append(success_rate)
                    # Fairness would require per-node statistics which would need simulator modification
                
                # Store average metrics
                metrics['throughput'][key].append(np.mean(throughputs))
                metrics['collisions'][key].append(np.mean(collisions))
                metrics['latency'][key].append(np.mean(latencies))
                metrics['efficiency'][key].append(np.mean(efficiencies))
                metrics['success_rate'][key].append(np.mean(success_rates))
        
        # Plot all metrics
        self._plot_metrics(metrics, node_counts)
        
        return metrics
    
    def _plot_metrics(self, metrics, node_counts):
        """Plot all collected metrics"""
        # Create a figure with multiple subplots
        plt.figure(figsize=(18, 12))
        
        # Throughput plot
        plt.subplot(2, 3, 1)
        plt.plot(node_counts, metrics['throughput']['without_rts'], 'r-', label='Without RTS/CTS')
        plt.plot(node_counts, metrics['throughput']['with_rts'], 'b-', label='With RTS/CTS')
        plt.xlabel("Number of Nodes")
        plt.ylabel("Throughput (packets/unit time)")
        plt.title("Network Throughput")
        plt.legend()
        plt.grid(True)
        
        # Collisions plot
        plt.subplot(2, 3, 2)
        plt.plot(node_counts, metrics['collisions']['without_rts'], 'r-', label='Without RTS/CTS')
        plt.plot(node_counts, metrics['collisions']['with_rts'], 'b-', label='With RTS/CTS')
        plt.xlabel("Number of Nodes")
        plt.ylabel("Number of Collisions")
        plt.title("Collision Count")
        plt.legend()
        plt.grid(True)
        
        # Latency plot
        plt.subplot(2, 3, 3)
        plt.plot(node_counts, metrics['latency']['without_rts'], 'r-', label='Without RTS/CTS')
        plt.plot(node_counts, metrics['latency']['with_rts'], 'b-', label='With RTS/CTS')
        plt.xlabel("Number of Nodes")
        plt.ylabel("Average Latency (time units)")
        plt.title("Packet Transmission Latency")
        plt.legend()
        plt.grid(True)
        
        # Efficiency plot
        plt.subplot(2, 3, 4)
        plt.plot(node_counts, metrics['efficiency']['without_rts'], 'r-', label='Without RTS/CTS')
        plt.plot(node_counts, metrics['efficiency']['with_rts'], 'b-', label='With RTS/CTS')
        plt.xlabel("Number of Nodes")
        plt.ylabel("Efficiency Ratio")
        plt.title("Channel Utilization Efficiency")
        plt.legend()
        plt.grid(True)
        
        # Success rate plot
        plt.subplot(2, 3, 5)
        plt.plot(node_counts, metrics['success_rate']['without_rts'], 'r-', label='Without RTS/CTS')
        plt.plot(node_counts, metrics['success_rate']['with_rts'], 'b-', label='With RTS/CTS')
        plt.xlabel("Number of Nodes")
        plt.ylabel("Success Rate")
        plt.title("Transmission Success Rate")
        plt.legend()
        plt.grid(True)
        
        plt.tight_layout()
        plt.show()
    
    def analyze_backoff_behavior(self, node_count=10, iterations=50):
        """Analyze how backoff algorithm affects performance"""
        print("ANALYZING BACKOFF BEHAVIOR...")
        
        original_attempt_limit = self.simulator.ATTEMPT_LIMIT
        attempt_limits = [5, 10, 15, 20]
        
        metrics = {
            'throughput': [],
            'collisions': [],
            'latency': []
        }
        
        self.simulator.NUMBER_OF_NODES = node_count
        
        for limit in attempt_limits:
            print(f"\nTesting with attempt limit {limit}")
            self.simulator.ATTEMPT_LIMIT = limit
            
            throughputs = []
            collisions = []
            latencies = []
            
            for _ in range(iterations):
                self.simulator.reset_stats()
                env = simpy.Environment()
                self.simulator.channel = simpy.Resource(env, 1)
                env.process(self.simulator.setup(env))
                env.run(until=self.simulator.SIM_TIME)
                
                if self.simulator.LAST_TIME > 0:
                    throughput = self.simulator.PACKETS / self.simulator.LAST_TIME
                    latency = self.simulator.LAST_TIME / self.simulator.PACKETS if self.simulator.PACKETS > 0 else 0
                else:
                    throughput = 0
                    latency = 0
                
                throughputs.append(throughput)
                collisions.append(self.simulator.COLLISIONS)
                latencies.append(latency)
            
            metrics['throughput'].append(np.mean(throughputs))
            metrics['collisions'].append(np.mean(collisions))
            metrics['latency'].append(np.mean(latencies))
        
        # Restore original setting
        self.simulator.ATTEMPT_LIMIT = original_attempt_limit
        
        # Plot results
        plt.figure(figsize=(15, 5))
        
        plt.subplot(1, 3, 1)
        plt.plot(attempt_limits, metrics['throughput'], 'g-')
        plt.xlabel("Attempt Limit")
        plt.ylabel("Throughput")
        plt.title("Throughput vs Attempt Limit")
        plt.grid(True)
        
        plt.subplot(1, 3, 2)
        plt.plot(attempt_limits, metrics['collisions'], 'g-')
        plt.xlabel("Attempt Limit")
        plt.ylabel("Collisions")
        plt.title("Collisions vs Attempt Limit")
        plt.grid(True)
        
        plt.subplot(1, 3, 3)
        plt.plot(attempt_limits, metrics['latency'], 'g-')
        plt.xlabel("Attempt Limit")
        plt.ylabel("Latency")
        plt.title("Latency vs Attempt Limit")
        plt.grid(True)
        
        plt.tight_layout()
        plt.show()
        
        return metrics


# Main execution with enhanced analysis
if __name__ == "__main__":
    simulator = CSMA_CA_Simulator()
    analyzer = EnhancedAnalyzer(simulator)
    
    print("Starting basic simulation...")
    simulator.simple_run()
    
    print("\nRunning comprehensive analysis...")
    analyzer.comprehensive_analysis(max_nodes=20, iterations=20)
    
    print("\nAnalyzing backoff behavior...")
    analyzer.analyze_backoff_behavior()
    
    print("\nAnalyzing RTS/CTS performance...")
    simulator.analyze_cts_rts()
    
    print("\nAnalyzing collision behavior...")
    simulator.analyze_collisions()
    
    print("\nAnalyzing coverage area impact...")
    simulator.analyze_coverage_area()